# CHANGE THESE CONFIGS, THEN UPDATE MODEL INITIALIZATION

In [46]:
MODEL_PATH = "./models/dqn/dqn"
TENSORBOARD_LOG_DIR = "./models/dqn/logs"
IMAGE_TAG = "dqn_rewards"
IMAGE_DIR = "./images/dqn"
RUNS_FILE = "./models/dqn/dqn_success_runs.pkl"

#### ENV CONFIGS ####
CONFIG = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 15,  # Number of other vehicles to observe
        "features": ["presence", "x", "y", "vx", "vy"],  # Observe position and velocity
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-10, 10],
            "vy": [-10, 10]
        },
        "absolute": False,
        "clip": False,
        "normalize": False
    },
    "action": {
        "type": "DiscreteMetaAction",  # Keep simple, 5 discrete actions
    },
    "simulation_frequency": 10,
    "policy_frequency": 10,
    "destination": "o1",
    "initial_vehicle_count": 20,
    "spawn_probability": 0.8,
    "ego_spacing": 25,
    "initial_lane_id": None,
    "controlled_vehicles": 1,
    "duration": 15,  # seconds
    "vehicles_density": 1.0,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.6],
    "scaling": 5.5 * 1.3,
    "normalize_reward": False
}

## Imports

In [47]:
import time
import torch
import pickle
import random
import gymnasium as gym

from fvcore.nn import FlopCountAnalysis

from stable_baselines3 import A2C, PPO, SAC, DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import ProgressBarCallback

from custom_intersection_env import CustomIntersectionEnv
from custom_training_callback import RewardTrackingCallback

## Register Env with Gymnasium

In [48]:
gym.envs.registration.register(
    id='custom-intersection-v0',
    entry_point='custom_intersection_env:CustomIntersectionEnv',
)

c:\Users\igor\Documents\UCSD\CSE190_Ammanbdrolu\cse-190-final-project\.venv\Lib\site-packages\gymnasium\envs\registration.py:644: UserWarning: WARN: Overriding environment custom-intersection-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


## Create and Wrap Env

In [49]:
env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=CONFIG)
env = DummyVecEnv([lambda: env])

## UPDATE HERE: Set Up Correct Model

In [ ]:
# model = A2C(
#     "MlpPolicy",
#     env,
#     n_steps=5,
#     learning_rate=7e-4,
#     gamma=0.99,
#     gae_lambda=1.0,
#     ent_coef=0.01,
#     vf_coef=0.5,
#     max_grad_norm=0.5,
#     rms_prop_eps=1e-5,
#     use_rms_prop=True,
#     normalize_advantage=False,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device='cpu'
# )
print(torch.version.cuda)  # Should not be None
print(torch.backends.cudnn.enabled)  # Should be True if CUDA is usable
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print (device)
# chat
# model = DQN(
#     "MlpPolicy",
#     env,
#     learning_rate=1e-4,  # often smaller for DQN
#     buffer_size=15000,
#     learning_starts=1000,
#     batch_size=32,
#     tau=1.0,  # soft update coefficient
#     gamma=0.99,
#     train_freq=4,
#     target_update_interval=1000,
#     exploration_fraction=0.1,
#     exploration_final_eps=0.02,
#     max_grad_norm=10,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device=device
# )
# default
# model = DQN("MlpPolicy",
#     env,
#     learning_rate=1e-4,
#     gamma=0.99,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device=device)
# docs example https://highway-env.farama.org/quickstart/
model = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=1000,
              learning_starts=100,
              batch_size=32,
              gamma=0.8,
              train_freq=4,
              gradient_steps=1,
              target_update_interval=20,
              verbose=1,
              tensorboard_log=TENSORBOARD_LOG_DIR,
              device=device)

11.8
True
cuda
Using cuda device


## Training

In [ ]:
reward_callback = RewardTrackingCallback(
    tag=IMAGE_TAG,
    path_dir=IMAGE_DIR
)

destinations = ["o1", "o2", "o3"]
steps = [5000 for _ in destinations]

for dest, steps in zip(destinations, steps):
    config = CONFIG.copy()
    config["destination"] = dest  # Change destination for each training phase
    env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=config)
    env = DummyVecEnv([lambda: env])
    model.set_env(env)  # Update the model with the new environment
    model.learn(
        total_timesteps=steps,
        callback=[ProgressBarCallback(), reward_callback]
    )
    reward_callback.start_new_phase()
reward_callback.save_all_plot()

Logging to ./models/dqn/logs\DQN_15


Output()

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 126      |
|    time_elapsed     | 1        |
|    total_timesteps  | 215      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 1.85     |
|    n_updates        | 3        |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 111      |
|    time_elapsed     | 3        |
|    total_timesteps  | 387      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.255    |
|    n_updates        | 46       |
----------------------------------


Logging to ./models/dqn/logs\DQN_16


Output()

Logging to ./models/dqn/logs\DQN_17


Output()

### Save Model

In [39]:
model.save(MODEL_PATH)

## Evaluation

### Load Model

In [40]:
model = DQN.load(MODEL_PATH)

### Run Evaluation

In [41]:
collisions = 0
destination_arrivals = 0
success_count = 0
successful_flopcount = 0
episodes = 100

# Store successful runs for rendering
successful_runs = []

for eps in range(100):
    config = CONFIG.copy()
    config["destination"] = "o" + str(random.randint(1, 3))
    env = gym.make("custom-intersection-v0", render_mode='rgb_array', config=config)

    seed = random.randint(0, 10000)

    obs, _ = env.reset(seed=seed)
    episode_flops = 0
    done = False
    truncated = False
    episode_reward = 0
    trajectory = []

    while not (done or truncated):
        # Flop Counting
        input_tensor, _ = model.policy.obs_to_tensor(obs)
        flops = FlopCountAnalysis(model.policy, input_tensor)
        flops.unsupported_ops_warnings(False)
        flops = flops.total()
        episode_flops += flops

        action, _states = model.predict(obs, deterministic=True)
        trajectory.append((obs, action))  # Save for later render if successful
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward
    
    crashed = info.get("crashed", False)
    arrived = info.get("arrived", False)
    if crashed:
        collisions += 1
    if arrived:
        destination_arrivals += 1
    if (not crashed) and arrived:
        success_count += 1
        successful_flopcount += episode_flops
        successful_runs.append((seed, config.copy(), trajectory))

    print(f"Episode {eps + 1} finished, total reward: {episode_reward}, destination: {config['destination']}")
    time.sleep(1)

print(f"Total collisions: {collisions} out of {episodes} episodes")
print(f"Total destination arrivals: {destination_arrivals} out of {episodes} episodes")
if success_count > 0:
    print(f"FLOPS per successful episode: {successful_flopcount / success_count:.2}")
else:
    print("No successful episodes.")



The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 1 finished, total reward: -20.0, destination: o3


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 2 finished, total reward: 18.0, destination: o3


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

...

Episode 98 finished, total reward: -20.0, destination: o2


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 99 finished, total reward: -10.0, destination: o3


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 100 finished, total reward: -46.0, destination: o3
Total collisions: 73 out of 100 episodes
Total destination arrivals: 5 out of 100 episodes
FLOPS per successful episode: 7.6e+06


### Save Successful Runs

In [42]:
with open(RUNS_FILE, "wb") as f:
    pickle.dump(successful_runs, f)

### Load Successful Runs File

In [43]:
with open(RUNS_FILE, "rb") as f:
    successful_runs = pickle.load(f)

### Render Successful Episodes

In [45]:
for i, (seed, config, trajectory) in enumerate(successful_runs):
    print(f"\nRendering successful episode {i + 1}")
    env = gym.make("custom-intersection-v0", render_mode='human', config=config)
    obs, _ = env.reset(seed=seed)
    for obs, action in trajectory:
        env.step(action)
        env.render()
        time.sleep(0.05)
    env.close()


Rendering successful episode 1

Rendering successful episode 2

Rendering successful episode 3

Rendering successful episode 4

Rendering successful episode 5
